# Tutorial to retrieve soil data from ```owimetadatabase```

If you haven't already, you need to install the following packages in your Python environment or if you want to update them (mostly owimetadatabse-preprcoessor as it is frequently updated) (remove *%%capture* here if you have problems when installing):

In [ ]:
%%capture
%pip install python-dotenv==1.0.0
%pip install owimetadatabase_preprocessor

**Import**

Load necessary modules:

In [ ]:
import os
from owimetadatabase_preprocessor.soil import SoilAPI, SoilprofileProcessor
# To check software version
from importlib_metadata import version

**Version** ```owimetadatabase_preprocessor``` 

You can verify which version is being used in your current environment as shown 

In [ ]:
_v = version("owimetadatabase_preprocessor")
print(_v)

## 1. Credentials and helper functions 

For authorization, the recommended way is to store your access token securely for authentication locally as an environment variable (can be in *.env* file next to your code, e.g. *.env* file with *OWIMETA_STAGING_TOKEN=<your-token-here>* field). Otherwise, just copying it to the **TOKEN** variable also works (but be careful with sahring/publishing, delete it from the notebook before). 

To load it into variable securely with your *.env* file:

In [ ]:
TOKEN = os.getenv('OWIMETA_STAGING_TOKEN') 

<div class="alert alert-block alert-warning">
    Please note that you will need to change the <b>input arguments</b> in this notebook according to what you have access to, so it will actually provide an output.
</div>

Additionally let us define some helper functions for later:

In [ ]:
def show_attrs(class_object: object):
    print(f"{class_object.__class__.__name__} class attributes: {class_object.__dict__.keys()}")

def show_props(class_object: object):
    props = [prop for prop in vars(type(class_object)).keys() if isinstance(getattr(type(class_object), prop), property)]
    print(f"{class_object.__class__.__name__} class properties: {props}")

def show_methods(class_object: object):
    methods = [method for method in dir(class_object) if callable(getattr(class_object, method)) and not method.startswith('_')]
    print(f"{class_object.__class__.__name__} class methods: {methods}")

## 2. Handeling soil related data 

### 2.1. Data retrieval

Soil data can be retrieved through the dedicated api client through the ```SoilAPI``` class. To start, as usual, we first initiate the specific API object with your credentials.

In [ ]:
api_geo = SoilAPI(token=TOKEN)

Now, we can start working with it. Since it is essentially a lot of similar methods but for different types of data, here, we will show only a few of them as examples, the rest can be consulted in the [documentation](https://owi-lab.github.io/owimetadatabase-preprocessor/index.html). 

#### Geotechnical campaigns

To start, let's see what survey campaigns are available:

In [ ]:
survey_campaigns = api_geo.get_surveycampaigns()
survey_campaigns["data"]

For this example, we can use the data above next, to use the method allowing to look into a specific campaign if you have the information (name) already.

In [ ]:
project = survey_campaigns["data"]["projectsite_name"].iloc[0]
survey_campaign = survey_campaigns["data"]["title"].iloc[0]

In [ ]:
survey_campaigns_details = api_geo.get_surveycampaign_detail(projectsite=project, campaign=survey_campaign)
survey_campaigns_details["data"]

#### Test locations

To show the closest locations based on longitude and latitude for a given radius, we run the following:

In [ ]:
close_loc = api_geo.get_proximity_testlocations(latitude=51.5, longitude=2.8, radius=10.0)
if close_loc["exists"]:
    display(close_loc["data"])
else:
    print("No test locations found within the radius or/and at this location.")

<div class="alert alert-block alert-info">
    Please note the outcome of this search depends on to what you have access to. In case of failure to find anything it usually
    returns an empty dataframe and False exists value. Additionaly, you can use specific method to get only the closest location using
    the above input data as well.
</div>

Some types of data allow for explicit checking if the specific element exists, e.g. for test locations:

In [ ]:
close_loc = api_geo.testlocation_exists(projectsite="Nobelwind", campaign="Borehole campaign", location="CPT-888")
if close_loc:
    print(f"Test location exists, id={close_loc["id"]}")
else:
    print(f"No test location according search criteria exists")

#### In-situ tests

Results from CPT, including post-processed CPT measurements can be found as follows:

In [ ]:
res = api_geo.get_cpttest_detail(insitutest="CPT-888")
display(res["id"])
display(res["insitutestsummary"])
display(res["rawdata"])
display(res["processeddata"])

#### Soil profiles

Soil tests conducted in geotechnical studies serve to characterize the ground. Based on the available data, geotechnical engineers define a soil profile for a given location and scope. 

These profiles, if available, can be retrived in the database as shown below. Note that the user needs to know beforehand ```soilprofile```.

In [ ]:
soil_profile = api_geo.get_soilprofile_detail(
    projectsite="Nobelwind", 
    location="BBK04", 
    soilprofile='Lateral Robertson Best estimate')['soilprofile']

## 2.2. Data post-processing

### 2.1. Soil profile for SSI modeling

After retrieving a soil profile, it can be post-processed to ensure using ```SoilprofileProcessor``` it includes all necessary parameters for a specific geotechnical analysis within a given soil reaction framework. Below, the previously retrieved soil profile is further processed to obtain the required inputs for the PISA lateral design model.

#### Lateral 

In [ ]:
sbl = -30.0  # mLAT
pisa_sp = SoilprofileProcessor.lateral(soil_profile, option="pisa", mudline=sbl)
pisa_sp

Similarly, one can consult the available options, for a given loading conditions, either ```lateral``` or ```axial```, that are supported in the current version.

In [ ]:
SoilprofileProcessor.get_available_options(loading='lateral')

<div class="alert alert-block alert-warning">
    <b>Not all soil profiles stored in the database contain the necessary soil parameters for a given SSI model.</b> The preprocessor automatically manages this and provides an informative error when required information is missing, as shown below. 
</div>

In [ ]:
SoilprofileProcessor.lateral(soil_profile, option="apirp2geo", mudline=sbl)

#### Axial

<div class="alert alert-block alert-info">
    <b>Under development.</b>
</div>

## 2.3. Visualize

Plot functions for data visualization are included in the ```SoilPlot``` class which needs an instance of the current active ```SoilAPI``` class.

In [ ]:
from owimetadatabase_preprocessor.soil import SoilPlot

In [ ]:
plotter = SoilPlot(api_geo)

An example to plot test locations from provided data is shown below.

In [ ]:
plotter.plot_testlocations(projectsite="Nobelwind", campaign="Borehole")